# Hash table

La **tabla hash (hash table)** es una estructura de datos que relaciona parejas de datos: una llave y un valor (las llaves son únicas). Para poder acceder al **valor** necesitaremos consultar a la estructura usando la **llave**. Muchos lenguajes de programación contienen esta estructura ya implementada.

1. Python $\Rightarrow$ `<class 'dict'>`
2. C++ $\Rightarrow$ `std::unordered_map`
3. Java $\Rightarrow$ `HashMap` o `Hashtable`

Veamos el uso de una tabla hash en Python:

In [ ]:
# Creamos una tabla hash vacía

hash_table = {}
print(type(hash_table))

# Agregamos llaves con sus respectivos valores

hash_table['key1'] = 'value1'
hash_table['key2'] = 'value2'
hash_table['key3'] = 'value3'
hash_table['key4'] = 'value4'

# Accedemos a los valores por sus llaves

print(hash_table['key2'])
print(hash_table['key4'])

# Eliminamos una llave de la tabla hash

value = hash_table.pop('key3')
print('key3' in hash_table)

Las tablas hash tienen por lo general estas tres operaciones que vimos:

1. Agregar una llave
2. Acceder al valor de una llave
3. Eliminar un llave

Estas operaciones tienen un tiempo **promedio** de ejecución de $O(1)$. 

Ahora veamos cómo se construye y cómo funciona una tabla hash. Primero simulemos el funcionamiento de una tabla hash usando una lista:

In [ ]:
class NaiveHashTable:
    def __init__(self):
        self.table = []

    # Complexity: O(n)
    def add_element(self, key, value):
        index = -1

        for i, (k, _) in enumerate(self.table):
            if key == k:
                index = i

        if index == -1:
            self.table.append((key, value))
        else:
            self.table[index] = (key, value)

    # Complexity: O(n)
    def get_value(self, key):
        for k, v in self.table:
            if key == k:
                return v
            
    # Complexity: O(n)
    def del_element(self, key):
        index = -1

        for i, (k, _) in enumerate(self.table):
            if key == k:
                index = i

        if index != -1:
            self.table.pop(index)

hash_table = NaiveHashTable()

hash_table.add_element('key1', 'value1')
hash_table.add_element('key2', 'value2')
hash_table.add_element('key3', 'value3')
hash_table.add_element('key4', 'value4')

print(hash_table.get_value('key2'))
print(hash_table.get_value('key4'))

hash_table.del_element('key1')

print(hash_table.get_value('key1')) # None

Esta estructura que diseñamos simula el funcionamiento de una tabla hash, sin embargo, la complejidad computacional es $O(n)$. 

Ahora, construiremos una estructura más eficiente, pero vamos a considerar dos condiciones:
1. Las llaves son **números enteros (no negativos)**.
2. Las llaves no son números muy altos. En este caso, vamos a considerar que los valores son menores a $10^7$.

Con estas dos condiciones, podemos crear una tabla hash usando simplemente una lista:

In [ ]:
class HashTable_v0:
    def __init__(self, size):
        self.table = [None for _ in range(size)]

    # Complexity: O(1)
    def add_element(self, key, value):
        self.table[key] = value

    # Complexity: O(1)
    def get_value(self, key):
        return self.table[key]
    
    # Complexity: O(1)
    def del_element(self, key):
        self.table[key] = None

hash_table = HashTable_v0(10000000)

hash_table.add_element(4351, 'value1')
hash_table.add_element(63243, 'value2')
hash_table.add_element(73456, 'value3')
hash_table.add_element(4134, 'value4')

print(hash_table.get_value(4351))
print(hash_table.get_value(73456))

hash_table.del_element(63243)

print(hash_table.get_value(63243)) # None

Observamos, que con estas dos condiciones, simplemente creamos una lista de tamaño $10^7$ para simular una tabla hash. Usamos el hecho de que acceder un elemento dado el índice en una lista, tiene una complejidad $O(1)$.

Para que funcione esta solución, las llaves deben de cumplir las dos condiciones anteriores. **Intentaremos forzar que toda llave cumpla ambas condiciones**. 

## Las llaves son números enteros

Es importante que las llaves sean números, pues las usaremos como índices para acceder al valor de una lista. Para esto, usaremos una **función hash**. Una función hash es un algoritmo que toma una entrada (puede ser de cualquier tipo, por ejemplo un string) y produce un número fijo de salida, generalmente un número entero, que se usa como índice para almacenar y recuperar datos en una tabla hash. En síntesis, esta función hash nos permite "convertir" una entrada (llave en este caso) en un número entero (no negativo). Además, si dos objetos distintos son pasados por la función, se deben obtener resultados también distintos.

In [ ]:
# Este es un ejemplo de función hash. Sin embargo, no es una buena implementación, pues para cualquier objeto siempre retornará 1

def dummy_hash_function(anything):
    return 1

print(dummy_hash_function('key1'))
print(dummy_hash_function('key1'))
print(dummy_hash_function('key1'))

In [ ]:
# Python cuenta con una función hash implementada: hash
# Observamos que para distintas llaves se generan distintos resultados

print(hash('key1'))
print(hash('key2'))
print(hash('key3'))

Con esto, ya podemos quitar la primera condición. Podemos transformar cualquier llave en un número entero no negativo usando la función `hash` de Python.

## Las llaves no son números muy altos

La segunda condición es que las llaves no pueden ser números muy altos, pues estos serán usados como índices para acceder a elementos de una lista. Por tanto, si creamos una lista de tamaño $n$ las llaves deben ser números entre $0$ y $n-1$. Debido a que no contamos con memoria infinita, una lista debe de contenter un tamaño razonable. Observamos que la función `hash` de Python retorna valores numéricos muy altos pero nosotros queremos que pertenezca al rango $[0, n-1]$. Para esto, simplemente obtenemos el residuo entre el valor obtenido por la función y $n$:

In [ ]:
n = 100000 # Supongamos que nuestra tabla hash contiene n = 10^5 elementos

print(hash('key1') % n)
print(hash('key2') % n)
print(hash('key3') % n)

## Implementación de una tabla hash

Ahora que ya vimos como podemos transformar nuestra llave para que cumpla ambas condiciones, unimos todo esto en una estrctura para crear una tabla hash eficiente:

In [ ]:
class HashTable_v1:
    def __init__(self, n):
        self.n = n
        self.table = [None for _ in range(n)]

    # Funcion hash: O(1)
    def hash_function(self, key):
        hash_value = hash(key) % self.n

        # Puede suceder el caso donde el valor que retorna hash() sea negativo
        # Para resolver esto, simplemente le sumamos n y aseguramos que el resultado será positivo
        if hash_value < 0: hash_value += self.n

        return hash_value

    # Complexity: O(1)
    def add_element(self, key, value):
        key_index = self.hash_function(key)
        self.table[key_index] = value

    # Complexity: O(1)
    def get_value(self, key):
        key_index = self.hash_function(key)
        return self.table[key_index]
    
    # Complexity: O(1)
    def del_element(self, key):
        key_index = self.hash_function(key)
        self.table[key_index] = None

n = 10000
hash_table = HashTable_v1(n)

hash_table.add_element('key1', 'value1')
hash_table.add_element('key2', 'value2')
hash_table.add_element('key3', 'value3')
hash_table.add_element('key4', 'value4')

print(hash_table.get_value('key2'))
print(hash_table.get_value('key4'))

hash_table.del_element('key1')

print(hash_table.get_value('key1')) # None

Ahora solo nos queda una problemática que resolver: las colisiones. Cuando dos objetos distintos son pasados por una función hash y se obtiene un mismo resultado. En nuestra tabla hash, se pueden dar colisiones y esto hará fallar el funcionamiento de nuestra estructura.

Veamos el siguiente caso:

In [ ]:
n = 10000
hash_table = HashTable_v1(n)

hash_table.add_element('aab', 'value1')
hash_table.add_element('aem', 'value2')

print(hash_table.get_value('aab')) # Se obtiene value2
print(hash_table.get_value('aem')) # Se obtiene value2

Hemos añadido la llave 'aae' con el valor 'value1' a nuestra tabla hash. Sin embargo, cuando consultamos por su valor, nos resulta 'value2'. ¿Por qué sucede esto? Por que ambas llaves generan el mismo resultado cuando son evaluadas por la función hash (es decir, sufren una colisión):

In [ ]:
hash_value_1 = hash_table.hash_function('aab')
hash_value_2 = hash_table.hash_function('aem')

print(hash_value_1) # 3325
print(hash_value_2) # 3325

Esto provoca que en nuestra estructura, los valores se sobreescriban y se pierda el valor de la llave 'aae'.

Una posible solución sería cambiar el valor de $n$ (ahora a 10^5):

In [ ]:
n = 100000
hash_table = HashTable_v1(n)

hash_table.add_element('aab', 'value1')
hash_table.add_element('aem', 'value2')

print(hash_table.get_value('aab')) 
print(hash_table.get_value('aem')) 

In [ ]:
hash_value_1 = hash_table.hash_function('aab')
hash_value_2 = hash_table.hash_function('aem')

print(hash_value_1) # 33325
print(hash_value_2) # 93325

Pero nuevamente, podemos encontrar dos valores que colisionen y hacer fallar a nuestra estructura:

In [ ]:
hash_table.add_element('afb', 'value3')
hash_table.add_element('ari', 'value4')

print(hash_table.get_value('afb')) 
print(hash_table.get_value('ari')) 

In [ ]:
hash_value_1 = hash_table.hash_function('afb')
hash_value_2 = hash_table.hash_function('ari')

print(hash_value_1) # 67719
print(hash_value_2) # 67719

Y así, para cada valor de $n$ podemos encontrar una colisión. De echo, la siguiente función encuentra una colisión para cualquier valor de $n$:

In [ ]:
def find_collision(n):
    hash_table = HashTable_v1(n)

    colissions = {}

    abc = 'abcdefghijklmnopqrstuvwxyz'

    for c1 in abc:
        for c2 in abc:
            for c3 in abc:
                s = c1 + c2 + c3
                h = hash_table.hash_function(s)

                if h not in colissions:
                    colissions[h] = []

                colissions[h].append(s)

                if len(colissions[h]) > 1:
                    return colissions[h][0], colissions[h][1]
                    
key1, key2 = find_collision(10000)
print(key1, key2)

key1, key2 = find_collision(100000)
print(key1, key2)

Ahora contemplaremos la posibilidad de que existan colisiones en mi tabla hash. Ahora guardaremos una lista para cada índice de la lista. Si existen varios objetos que apuntan a una misma posición, todos sus valores serán guardados en una lista:

In [ ]:
class HashTable_v2:
    def __init__(self, n):
        self.n = n
        self.table = [[] for _ in range(n)]

    # Funcion hash: O(1)
    def hash_function(self, key):
        hash_value = hash(key) % self.n

        # Puede suceder el caso donde el valor que retorna hash() sea negativo
        # Para resolver esto, simplemente le sumamos n y aseguramos que el resultado será positivo
        if hash_value < 0: hash_value += self.n

        return hash_value

    # Complexity: O(1) => Caso promedio, O(n) => Peor caso
    def add_element(self, key, value):
        key_index = self.hash_function(key)

        index = -1
        for i, (k, _) in enumerate(self.table[key_index]):
            if key == k:
                index = i

        if index == -1:
            self.table[key_index].append((key, value))
        else:
            self.table[key_index][index] = (key, value)

    # Complexity: O(1) => Caso promedio, O(n) => Peor caso
    def get_value(self, key):
        key_index = self.hash_function(key)

        for k, v in self.table[key_index]:
            if key == k:
                return v
            
        return None
    
    # Complexity: O(1) => Caso promedio, O(n) => Peor caso
    def del_element(self, key):
        key_index = self.hash_function(key)

        index = -1

        for i, (k, _) in enumerate(self.table[key_index]):
            if key == k:
                index = i

        if index != -1:
            self.table[key_index].pop(index)



Podemos darnos cuenta que esta nueva implementación es una combinación de `NaiveHashTable` y `HashTable_v0`. En el caso promedio, las operaciones de esta estructura solo tomará $O(1)$. Sin embargo, en el peor de los casos (cuenta la cantidad de soluciones sea considerable) la complejidad computacional será lineal ($O(n)$).

In [ ]:
n = 10000
hash_table = HashTable_v2(n)

hash_table.add_element('aab', 'value1')
hash_table.add_element('aem', 'value2')
hash_table.add_element('key1', 'value3')
hash_table.add_element('key2', 'value4')

print(hash_table.get_value('aab')) 
print(hash_table.get_value('aem')) 
print(hash_table.get_value('key1')) 
print(hash_table.get_value('key2')) 